<a href="https://colab.research.google.com/github/sigvehaug/DSF-DCBP/blob/main/02_Intro_Pandas_Numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas and Numpy

## 1. Why Pandas and NumPy?

Python has built-in data structures like lists and dictionaries, but they are not optimized for fast computation on large numerical tables. Two libraries fill this gap:

* **NumPy** provides fast *arrays* and numerical operations.
* **Pandas** provides higher-level tabular structures such as the **DataFrame**, built on top of NumPy.

In this notebook we will:

1. Load the polymer dataset (`train.csv`) from this [Kaggle competition](https://www.kaggle.com/competitions/neurips-open-polymer-prediction-2025/data) into a Pandas DataFrame
2. Inspect its structure and data types
3. Handle missing values
4. Compute simple summary statistics
5. Create basic visualizations
6. Peek under the hood: Pandas columns are NumPy arrays

---

## 2. Imports

```python
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
```

---

## 3. Loading the dataset

1. Download the data from the [Ilias page](https://ilias.unibe.ch/ilias.php?baseClass=ilrepositorygui&cmdNode=zo:ok&cmdClass=ilObjFileGUI&cmd=sendfile&ref_id=3663532).
2. Extract all files into a 'data' folder. Where you place the folder is important and changes its path!

### Option A — Load from local file

1. Upload it to your notebook environment
2. Use:

```python
# Adjust path if needed (e.g., "/data/train.csv" in Colab)
path = "data/train.csv"
df = pd.read_csv(path)

df.head()
```

### Option B — Load on Google Colab

```python
import os
try:
    from google.colab import drive
    drive.mount('/content/drive')

    path = "/content/drive/MyDrive/DSF-DCBP/data/train.csv"  # adjust based on your file location in Google Drive
    df = pd.read_csv(path)
    df.head()
except ImportError:
    print("Not running on Colab, load from local file system.")
```

---

## 4. First look: what did we load?

```python
type(df)
```

A Pandas **DataFrame** is a 2D table with:

* rows (observations)
* columns (variables)
* an index (row labels)

Show shape (rows, columns):

```python
df.shape
```

Show column names:

```python
df.columns
```

---

## 5. Dataset documentation (columns)

Each row corresponds to a polymer structure (as a SMILES string) with several target properties.

* `id`: unique identifier
* `SMILES`: polymer repeating unit encoded as text (SMILES)
* `Tg`: glass transition temperature (°C)
* `FFV`: fractional free volume
* `Tc`: thermal conductivity (W / m·K)
* `Density`: density (scaled; your note says `g·cm⁻3 × 10⁻3`)
* `Rg`: radius of gyration

---

## 6. Data types and missing values

### DataFrame overview

```python
df.info()
```

Key things to notice:

* Which columns are numeric?
* Which are text?
* How many missing values per column?

### Count missing values explicitly

```python
df.isna().sum() # if you want to also sort them: .sort_values(ascending=False)
```

---

## 7. Basic statistics (numeric columns)

```python
df.describe()
```

---

## 8. Accessing columns

Extract a single column (Series):

```python
tc = df["Tc"]
type(tc), tc.head()
```

Extract multiple columns (DataFrame):

```python
targets = df[["Tg", "FFV", "Tc", "Density", "Rg"]]
targets.head()
```

---

## 9. Under the hood: Pandas uses NumPy arrays

A Pandas Series stores its values in a NumPy array:

```python
tc_values = df["Tc"].values
type(tc_values), tc_values[:10]
```

---

## 10. Cleaning: select rows with at least one target available

In this dataset, many targets are missing for many rows. It’s useful to select rows with at least one property measured.

```python
target_cols = ["Tg", "FFV", "Tc", "Density", "Rg"]

df_any_target = df.dropna(subset=target_cols, how="all") # drop only if 'all' values in those column are missing
df_any_target.shape
```

Also useful: rows with **all targets present**:

```python
df_all_targets = df.dropna(subset=target_cols, how="any") #  if 'any' of the values in those columns are missing, drop that row
df_all_targets.shape
```

---

## 11. Simple feature engineering from SMILES

We can compute simple text-based features:

* SMILES length

```python
df["smiles_len"] = df["SMILES"].astype(str).str.len()
```

Or more complex features:
* count of specific characters (rings, branches, aromatic atoms)

```python
df["count_aromatic_c"] = df["SMILES"].astype(str).str.count("c")
df["count_branch"] = df["SMILES"].astype(str).str.count(r"\(")
df["count_ring_digits"] = df["SMILES"].astype(str).str.count(r"[0-9]")
df["count_star"] = df["SMILES"].astype(str).str.count(r"\*")

df[["SMILES", "smiles_len", "count_aromatic_c", "count_branch", "count_ring_digits", "count_star"]].head()
```

# Exercise

1. In the Data folder on Ilias there is an additional folder called 'train_supplement'. Choose one of the datasets in there.
    * If you use Google Colab, upload that file to a folder on your google drive. If you don't have a google account, make it now.
    * If you are working locally, download it and find its path.
    Look at the file by clicking on it. Does it contain the same columns of the 'train.csv' dataset?

2. Load the dataset as we did before for the 'train.csv' dataset

3. Study this dataset a bit with df.info() and df.describe() methods. How big is the dataset: How many rows, how many columns, how much space in the memory?

4. Which column has the most missing values among the targets, if any?

5. Plot some columns with an histogram

---

# Takeaways

* Pandas DataFrames are the standard for tabular data in Python.
* Columns are backed by NumPy arrays, enabling fast computations.
* Real datasets have missing values: you must inspect and handle them.
* You can create useful numeric features from currently existing ones.